#### Adventure Works Classification Challenge

First step - confirm we can connect to the service that we have deployed:

In [9]:
from azureml.core import Workspace
from azureml.core import Webservice
from azureml.core import Model

#Load workspace from default config
ws = Workspace.from_config(path=".file-path/ws_config.json")

#Load services available in this workspace
services = Webservice.list(ws)
classification_service = services[0]
print("Scoring URI: " + classification_service.scoring_uri)

#Authentication is disabled by default, but run this if you have it enabled
#print(classification_service.get_keys())

Scoring URI: http://0f937284-a94e-4c1d-8f14-3fbd026b6211.westus.azurecontainer.io/score


Okay, that's a decent start! Now let's set up a simple JSON object that will represent a new entry to make a prediction on. We populate the set of features that we trained the model on, then send it up to the scoring endpoint. We should expect a Yes/No classification to be returned, for whether this person will buy a bike or not.

In [10]:
data = { "data":
            [
                [
                    'Fremont',         #City
                    'California',      #StateProvinceName
                    'United States',   #CountryRegionName
                    '1/05/1945',       #BirthDate
                    'Bachelors',       #Education
                    'Management',      #Occupation
                    'F',               #Gender
                    'S',               #MaritalStatus
                    0,                 #HomeOwnerFlag
                    2,                 #NumberCarsOwned
                    0,                 #NumberChildrenAtHome
                    5,                 #TotalChildren
                    86931              #YearlyIncome
                ]          
            ]
        }

Let's see what we get:

In [4]:
import requests
import json

def get_response(request_data):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(classification_service.scoring_uri, request_data, headers=headers)
    return response

response = get_response(json.dumps(data))
response_data = json.loads(json.loads(response.text)) #double up json.loads as first result is still just a string
print(response_data['result'])

['0']


Well how cool is that?? We got a '0' response back, which is a prediction that this customer will not buy a bike.

How about if we want to make more than one prediction at a time? Luckily for us, we have been supplied with a CSV file containing a set of observations that the model has not seen before. The CSV has already been mangled so that the feature columns we ignored during training have been removed.

In [19]:
#Load rows from test set in CSV. Convert to lists, serialise as JSON, and send it off.
import pandas as pd

test_values = pd.read_csv('AW_Classification_Test.csv')
test_data = test_values.values.tolist()[0:5] #Let's just predict the first five entries

print("Request preview:")
print(test_data)

data = { "data": test_data }
request_data = json.dumps(data)
response = get_response(request_data)
response_data = json.loads(json.loads(response.text))

print("\n")
print("Predictions")
print(response_data['result'])

#Save the result to an output CSV file (will be created if it does not already exist)
#pd.DataFrame(response_data['result']).to_csv("AW_Classification_Results.csv", index=None)

Request preview:
[['Fremont', 'California', 'United States', '1/05/1945', 'Bachelors ', 'Management', 'F', 'S', 0, 2, 0, 5, 86931], ['Haney', 'British Columbia', 'Canada', '10/04/1964', 'Bachelors ', 'Skilled Manual', 'M', 'M', 1, 2, 2, 4, 100125], ['Burbank', 'California', 'United States', '1/12/1934', 'Graduate Degree', 'Management', 'F', 'M', 1, 2, 0, 4, 103985], ['Burlingame', 'California', 'United States', '9/22/1958', 'Graduate Degree', 'Skilled Manual', 'M', 'M', 1, 0, 0, 4, 127161], ['Dunkerque', 'Nord', 'France', '3/19/1965', 'High School', 'Manual', 'F', 'M', 1, 1, 2, 2, 21876]]


Predictions
['0', '1', '0', '0', '0']
